In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from preprocess_our_merfish import read_adata, preprocess

In [16]:
adata = read_adata(file_path="./our_data_LC_NE/adata_mer.h5ad")
adata_output = preprocess(adata)

Loaded sequence dictionary from /home/ec2-user/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.cdna.all.fa.gz.pickle
Loaded sequence dictionary from /home/ec2-user/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.ncrna.fa.gz.pickle
Loaded sequence dictionary from /home/ec2-user/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.pep.all.fa.gz.pickle
Starting validation...


total invalid 1, from 315 0.31746031746031744 %
new data size (3227, 314)


ERROR: Could not infer organism from feature ID 'NA' in 'raw.var', make sure it is a valid ID.
Validation complete in 0:00:00.503930 with status is_valid=False


In [1]:
import scanpy as sc
import sys

import os 
import sys
import pandas as pd
import scanpy as sc
from scipy.sparse import csr_matrix

from nicheformer.data.constants import DefaultPaths, ObsConstants, UnsConstants, VarConstants, AssayOntologyTermId, SexOntologyTermId, OrganismOntologyTermId, TissueOntologyTermId, SuspensionTypeId
from nicheformer.data.validate import validate

In [2]:
from cellxgene_schema.ontology import GeneChecker, SupportedOrganisms
from cellxgene_schema.utils import enforce_canonical_format
from cellxgene_schema.validate import Validator as CellxgeneValidator
from cellxgene_schema.validate import logger
from cellxgene_schema.write_labels import AnnDataLabelAppender as CellxgeneAnnDataLabelAppender

In [3]:
DefaultPaths.SPATIAL

'/home/ec2-user/SageMaker/test_data/spatial/'

In [4]:
#if len(sys.argv)==1:
path = DefaultPaths.SPATIAL
#else:
#    path = sys.argv[1]

raw_path = f"{path}"
preprocessed_path = f"{path}/preprocessed"

if not os.path.exists(f"{preprocessed_path}"):
    os.mkdir(f"{preprocessed_path}")

adata = sc.read_10x_h5(f"{raw_path}/Xenium_Preview_Human_Non_diseased_Lung_With_Add_on_FFPE_outs/cell_feature_matrix.h5")

adata.obs = pd.read_csv(
    f"{raw_path}/Xenium_Preview_Human_Non_diseased_Lung_With_Add_on_FFPE_outs/cells.csv.gz", index_col=0
)

adata.var = adata.var.reset_index().rename(columns={'index': 'gene_name'}).set_index('gene_ids')
adata.var.index.name = None

# setting MERFISH for now until an official ontology term is released
assay = str(AssayOntologyTermId.MERFISH_SPATIAL.value)
sex = str(SexOntologyTermId.UNKNOWN.value)
organism = str(OrganismOntologyTermId.HUMAN.value)
organism_validator = "human"
tissue = str(TissueOntologyTermId.LUNG.value)
suspension_type = str(SuspensionTypeId.SPATIAL.value)
tissue_type = "tissue"

adata.X = csr_matrix(adata.X)
adata.obs[ObsConstants.SPATIAL_X] = adata.obs['x_centroid']
adata.obs[ObsConstants.SPATIAL_Y] = adata.obs['y_centroid']
adata.obs[ObsConstants.ASSAY_ONTOLOGY_TERM_ID] = pd.Categorical([assay for i in range(len(adata))])
adata.obs[ObsConstants.SEX_ONTOLOGY_TERM_ID] = pd.Categorical([sex for i in range(len(adata))])
adata.obs[ObsConstants.ORGANISM_ONTOLOGY_TERM_ID] = pd.Categorical([organism for i in range(len(adata))])
adata.obs[ObsConstants.TISSUE_ONTOLOGY_TERM_ID] = pd.Categorical([tissue for i in range(len(adata))])
adata.obs[ObsConstants.SUSPENSION_TYPE] = pd.Categorical([suspension_type for i in range(len(adata))])

adata.obs[ObsConstants.DONOR_ID] = pd.Categorical(['Xenium_Preview_Human_Non_diseased_Lung_With_Add_on_FFPE_outs' for i in range(len(adata))])
adata.obs[ObsConstants.CONDITION_ID] = pd.Categorical(['non diseased' for i in range(len(adata))])
adata.obs[ObsConstants.TISSUE_TYPE] = pd.Categorical([tissue_type for i in range(len(adata))])

adata.uns[UnsConstants.TITLE] = "Xenium_Preview_Human_Non_diseased_Lung_With_Add_on_FFPE_outs"
adata.var[VarConstants.FEATURE_IS_FILTERED] = False

adata.obs[ObsConstants.LIBRARY_KEY] = pd.Categorical(['section' for i in range(len(adata))])

adata_output, valid, errors, is_seurat_convertible = validate(adata, organism=organism_validator)


Starting validation...
Validation complete in 0:00:17.381665 with status is_valid=True
Writing labels
enforce canonical format in X
H5AD label writing complete in 0:00:00.221493


--- enforce_canonical_format ---


In [5]:
adata_output.obs['assay'] = pd.Categorical(['Xenium' for i in range(len(adata_output))])
adata_output.obs[ObsConstants.ASSAY_ONTOLOGY_TERM_ID] = pd.Categorical(['no yet defined' for i in range(len(adata_output))])

adata_output.obs[ObsConstants.DATASET] = adata_output.uns['title']
adata_output.obs[ObsConstants.SPLIT] = 'train'
adata_output.obs[ObsConstants.NICHE] = 'nan'
adata_output.obs[ObsConstants.REGION] = 'nan'

adata_output.write(f"{preprocessed_path}/Xenium_Preview_Human_Non_diseased_Lung_With_Add_on_FFPE_outs.h5ad")

... storing 'dataset' as categorical
... storing 'nicheformer_split' as categorical
... storing 'niche' as categorical
... storing 'region' as categorical
... storing 'feature_types' as categorical
... storing 'genome' as categorical


In [6]:
adata.X.max()

66.0

In [46]:
from gseapy import Biomart
bm = Biomart()

2025-02-22 00:08:32,688 [WARNING] host www.ensembl.org is not reachable, try useast.ensembl.org 


In [47]:
from gseapy import Biomart
bm = Biomart()
# note the dataset and attribute names are different
m2h = bm.query(dataset='mmusculus_gene_ensembl',
               attributes=['ensembl_gene_id','external_gene_name',
                           'hsapiens_homolog_ensembl_gene',
                           'hsapiens_homolog_associated_gene_name'])

h2m = bm.query(dataset='hsapiens_gene_ensembl',
               attributes=['ensembl_gene_id','external_gene_name',
                           'mmusculus_homolog_ensembl_gene',
                           'mmusculus_homolog_associated_gene_name'])

2025-02-22 00:08:33,716 [WARNING] host www.ensembl.org is not reachable, try useast.ensembl.org 


In [49]:
# note the dataset and attribute names are different
m2h = bm.query(dataset='mmusculus_gene_ensembl',
               attributes=['ensembl_gene_id',
                           'hsapiens_homolog_ensembl_gene',
                           ])
m2h = m2h[m2h.hsapiens_homolog_ensembl_gene.notna()]